In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

# Input decision matrix (alternatives × criteria)
matrix = np.array([
    [0.134, 0.140, 0.160, 0.140],
    [0.199, 0.105, 0.304, 0.104],
    [0.156, 0.093, 0.061, 0.084],
    [0.021, 0.128, 0.050, 0.128],
    [0.103, 0.185, 0.041, 0.206],
    [0.178, 0.152, 0.157, 0.116],
    [0.021, 0.128, 0.069, 0.091],
    [0.188, 0.070, 0.160, 0.131]
])

# Site labels
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8']

# Criteria types: 'max' for benefit, 'min' for cost
criteria_types = ['max', 'max', 'max', 'max']

# Normalized weights
weights = np.array([0.15288888, 0.30888675, 0.45773133, 0.080493041])

def marcoss(matrix, weights, criteria_types, sites):
    # Step 1: Define AI and AAI
    ai = np.where(np.array(criteria_types) == 'max', 
                  np.max(matrix, axis=0), 
                  np.min(matrix, axis=0))
    
    aai = np.where(np.array(criteria_types) == 'max', 
                   np.min(matrix, axis=0), 
                   np.max(matrix, axis=0))
    
    # Step 2: Extend matrix with AI and AAI
    extended = np.vstack([matrix, ai, aai])

    # Step 3: Normalize extended matrix
    norm_extended = np.zeros_like(extended, dtype=float)
    for j in range(extended.shape[1]):
        if criteria_types[j] == 'max':
            norm_extended[:, j] = extended[:, j] / ai[j]
        else:
            norm_extended[:, j] = aai[j] / extended[:, j]
    
    # Step 4: Apply weights to normalized matrix
    weighted_extended = norm_extended * weights

    # Step 5: Compute S_i, S_AI, S_AAI
    S = weighted_extended.sum(axis=1)
    S_ai, S_aai = S[-2], S[-1]  # Last two rows are AI and AAI
    S_alts = S[:-2]             # Alternatives only

    # Step 6: Compute utility degrees
    K_plus = S_alts / S_ai
    K_minus = S_alts / S_aai

    # Step 7: Final utility function as compromise score
    utility = (K_plus + K_minus) / (
        1 + (1 - K_plus) / K_plus + (1 - K_minus) / K_minus
    )

    # Step 8: Rank alternatives
    ranks = rankdata(-utility, method='min')

    # Step 9: Return results
    return pd.DataFrame({
        'Site': sites,
        'Utility': utility,
        'Rank': ranks
    }).sort_values(by='Rank').reset_index(drop=True)

# Run MARCOS and show result
result_df = marcoss(matrix, weights, criteria_types, sites)
print(result_df)

    Site   Utility  Rank
0  Site2  9.190944     1
1  Site6  4.390221     2
2  Site1  3.623347     3
3  Site8  2.450622     4
4  Site5  2.175013     5
5  Site3  1.041639     6
6  Site7  0.857189     7
7  Site4  0.779898     8
